# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [179]:
# !pip install --upgrade gdown

# # Main link
# # !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
# # !gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

# # !unzip -q libriphone.zip
# !ls libriphone

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [180]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

TensorBoard Session

In [181]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [182]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    # x: concat_n, seq_len, feature_dim
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
        
    elif n > 0:
        left = x[n:]
        right = x[-1].repeat(n, 1)
        
    else:
        return x

    return torch.cat((left, right), dim=0)

# concatenate frame
def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    # feature 不需要改变位置 shift
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        # 这里 feat.size() == (seq_len, feature_dim)
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat) # seq_len

        # 这里 feat.size() == (seq_len, concat_n * feature_dim)
        feat = concat_feat(feat, concat_nframes)

        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        # 一个 audio 中，所有 frame（已经 concated）的 feature
        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    # 让 X 与 y 的长度保持一致
    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


preprocess_data_sequence

In [183]:
def preprocess_data_sequence(split, feat_dir, phone_path, train_ratio=0.8, random_seed=1213,crop_len=61, crop_step=30):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train':
        mode = 'train'
    elif split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    data = []
    # idx = 0
    short_data = 0
    for i, fname in tqdm(enumerate(usage_list), ncols=40):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat) # seq_len
        if cur_len + 1 - crop_len < 0:
            short_data += 1
            continue # 小于切割长度，则丢弃当前音频帧

        for j in range(0, cur_len + 1 - crop_len, crop_step):
            # TODO 最后一个小节可能不够分
            x = feat[j:j+crop_len, :]
            # X[idx: idx + crop_len, :] = x
            
            if mode == "train" or mode == "valid" :
                label = torch.LongTensor(label_dict[fname][j: j+crop_len])
                # y[idx: idx + crop_len] = label
                data.append({'x':x, 'y':label})
            else:
                data.append({'x':x})
   
            # idx += cur_len
    
    # 让 X 与 y 的长度保持一致
    # X = X[:idx, :]
    # if mode == 'train':
    #   y = y[:idx]

    # # print(f'[INFO] {split} set')
    # # print(X.shape)
    # if mode == 'train':
    # #   print(y.shape)
    #   return X, y
    # else:
    #   return X
    print(f'[INFO] {split} set')
    print("len_data:{}".format(len(data)))
    print("short_audio:{}".format(short_data))

    
    return data

preprocess_data_sequence for test data

In [184]:
def preprocess_data_sequence_test(split, feat_dir, phone_path, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')
    

    
    usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    data = []
    short_data = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))

        cur_len = len(feat) # seq_len

        # if cur_len + 1 - crop_len < 0:
        #     short_data += 1
        #     continue # 小于切割长度，则丢弃当前音频帧

        # for j in range(0, cur_len, crop_step):
        #     # TODO 最后一个小节可能不够分
        #     if j+crop_len <= cur_len - 1:
        #         x = feat[j:j+crop_len, :]
        #     else:
        #         x = feat[j:cur_len-1, :]   
            
        #     data.append({'x':x})    

        x = feat
        data.append({'x':x})  

                
   
    print(f'[INFO] {split} set')
    print("len_data:{}".format(len(data)))
    print("short_audio:{}".format(short_data))

    
    return data

# Dataset

In [185]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

class LibriDataset_whole(Dataset):
    def __init__(self, data):
        self.data = data
        self.crop_len = len(self.data[0]['x'])

    def __getitem__(self, idx):
            return self.data[idx]['x'], self.data[idx]['y']
            # return self.data[idx]['x']

    def __len__(self):
        return len(self.data) 

class LibriDataset_test(Dataset):
    def __init__(self, data):
        self.data = data
        self.crop_len = len(self.data[0]['x'])

    def __getitem__(self, idx):
            return self.data[idx]['x']

    def __len__(self):
        return len(self.data) 


# Model
Feel free to modify the structure of the model.

DNN network

In [186]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, output_dim),  
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )

    def forward(self, x):
        x = self.block(x)
        return x

class DNN_network(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256,dropout=0.5):
        super(DNN_network, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

RNN model in frame level

In [187]:

class RNN_network(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256,dropout=0.5):
        super(RNN_network, self).__init__()

        self.linear_input = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim)
        )
        
        self.RNN = nn.GRU(input_size=hidden_dim,hidden_size=hidden_dim,
                          num_layers=hidden_layers,batch_first=True,bidirectional=True,
                          dropout=dropout)
        
        self.output = nn.Sequential(
            nn.Linear(hidden_dim * 2,output_dim),
            # nn.ReLU()
        )


    def forward(self, x):
        x = self.linear_input(x)

        batch_len = x.shape[0]
        feature_len = x.shape[1]
        # print(batch_len,feature_len)
        x = x.view(batch_len,-1,feature_len)
        x, h_n = self.RNN(x)

        x_fd = h_n[-2, :, :]  # forward final state of final hidden layer  ==>  x:b,h
        x_bd = h_n[-1, :, :]  # backward final state of final hidden layer  ==>  x:b,h
        out = self.output(torch.cat([x_fd, x_bd], dim=-1))

        return out

RNN model in sequnce level  

In [188]:
class RNN_sequence_level(nn.Module):
    def __init__(self, input_dim=39, output_dim=41, hidden_layers=1, hidden_dim=256,dropout=0.5):
        super(RNN_sequence_level, self).__init__()
        
        self.RNN = nn.GRU(input_size=39, hidden_size=hidden_dim,
                          num_layers=hidden_layers, batch_first=True, bidirectional=True,
                          dropout=dropout)
        
        self.output = nn.Sequential(
            nn.Linear(hidden_dim * 2, output_dim),
        )

    def forward(self, x):
        # x: b, l, MFCC-dim
        # print(x.size())
        out, h_n = self.RNN(x) #out: b, l, 2*hidden_size
        output = self.output(out) #output:  b, l, output_dim
        output = output.view(-1, 41) #output: *, output_dim

        return output

# Hyper-parameters

In [189]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 31   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213          # random seed
batch_size = 512        # batch size 512
num_epoch = 50         # the number of training epoch, 10
learning_rate = 1e-3      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 8          # the number of hidden layers
hidden_dim = 256          # the hidden dim

dropout = 0.5

croplen = concat_nframes
cropstep = 30


# Dataloader

In [190]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
# train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
# val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
# test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)

train_data = preprocess_data_sequence(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone',  train_ratio=train_ratio, random_seed=seed, crop_len=croplen, crop_step=cropstep)
val_data = preprocess_data_sequence(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone',  train_ratio=train_ratio, random_seed=seed, crop_len=croplen, crop_step=cropstep)
# test_data = preprocess_data_sequence(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', crop_len= croplen, crop_step= croplen)

# get dataset
# train_set = LibriDataset(train_X, train_y)
# val_set = LibriDataset(val_X, val_y)
# test_set = LibriDataset(test_X, None)

train_set = LibriDataset_whole(train_data)
val_set = LibriDataset_whole(val_data)
# test_set = LibriDataset_test(test_data)


# remove raw feature to save memory
# del train_X, train_y, val_X, val_y, test_X

del train_data, val_data
# del train_data, val_data , test_data

gc.collect()

# get dataloader
# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=12, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 2571


764it [00:00, 1930.58it/s]

2571it [00:01, 1546.62it/s]


[INFO] train set
len_data:51637
short_audio:0
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:00, 2075.08it/s]


[INFO] val set
len_data:17171
short_audio:0
[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:00, 3071.86it/s]


[INFO] test set
len_data:16601
short_audio:0


# Training

In [191]:
# create model, define a loss function, and optimizer
model = RNN_sequence_level(input_dim=input_dim, hidden_layers=hidden_layers, 
                   hidden_dim=hidden_dim, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_acc = 0.0
step = 0

for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader, ncols=200)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        # RNN sequence, flatten batch 
        labels = labels.view(-1) 
        
        optimizer.zero_grad() 
        outputs = model(features)
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        step += 1
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

        # writer.add_scalar('Train/Acc', train_acc/len(train_set), step)
        # writer.add_scalar('Train/Loss', train_loss/len(train_loader), step)
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader, ncols=200)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
   
            outputs = model(features)
            # RNN sequence, flatten batch 
            labels = labels.view(-1) 
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()
            
    # writer.add_scalar('Val/Acc', val_acc/len(val_set), step)
    # writer.add_scalar('Val/Loss', val_loss/len(val_loader), step)

    mean_train_acc = train_acc/len(train_set)/croplen
    mean_train_loss = train_loss/len(train_loader) / croplen
    mean_val_acc = val_acc/len(val_set)/croplen
    mean_val_loss= val_loss/len(val_loader)


    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {mean_train_acc:3.5f} Loss: {mean_train_loss:3.5f} | Val Acc: {mean_val_acc:3.5f} loss: {mean_val_loss:3.5f}')

    # Tensorboard
    writer.add_scalar('Acc/Train', mean_train_acc, epoch)
    writer.add_scalar('Loss/Train', mean_train_loss, epoch)
       
    writer.add_scalar('Acc/Val', mean_val_acc, epoch)
    writer.add_scalar('Loss/Val', mean_val_loss, epoch)

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  6.98it/s]


[001/050] Train Acc: 0.46147 Loss: 0.05984 | Val Acc: 0.57031 loss: 1.41344
saving model with acc 17.67963


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  6.92it/s]


[002/050] Train Acc: 0.60049 Loss: 0.04224 | Val Acc: 0.64258 loss: 1.16726
saving model with acc 19.92004


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  6.96it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  6.96it/s]


[018/050] Train Acc: 0.79925 Loss: 0.02024 | Val Acc: 0.74407 loss: 0.85781
saving model with acc 23.06627


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  6.94it/s]


[019/050] Train Acc: 0.80403 Loss: 0.01971 | Val Acc: 0.74131 loss: 0.86956


  0%|                                                                                                                                                                            | 0/34 [00:00<?, ?it/s]

In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

57

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load Data

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
# test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_data = preprocess_data_sequence_test(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone')

# get dataset
test_set = LibriDataset_test(test_data)


# remove raw feature to save memory
del test_data
gc.collect()

# get dataloader
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=12, pin_memory=True)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:00, 3909.29it/s]


[INFO] test set
len_data:857
short_audio:0


In [ ]:
# load model
model = RNN_sequence_level(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 857/857 [01:00<00:00, 14.19it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))